# Import Section

In [ ]:
import numpy as np
from syncsweptsine import SyncSweep, LinearModel, FrfFilterKernel, HammersteinModel

## Import Virtual Sound Field

In [ ]:
# SOUND_FIELD = lambda x: 2. * x.copy()

from pysndfx import AudioEffectsChain
SOUND_FIELD = AudioEffectsChain().reverb()

## import Warning Controller

In [ ]:
from warnings import simplefilter
simplefilter('ignore')

## import Plotters

In [ ]:
from matplotlib import pylab as plt
import seaborn as sns

sns.set()
sns.set_context('paper')
sns.set_palette('Set2', 4)

# Define Global Variables

## Sample rate [Hz]

In [ ]:
RATE = 48000

## Total Time Length of Source [sec]

In [ ]:
LEN_T = 180

## Freq of Source [Hz]

In [ ]:
FREQ = 440

## Time-Axis

In [ ]:
t = np.linspace(0, LEN_T, LEN_T*RATE, dtype=np.float64)

## About FIR-Filter

### Filter Length

In [ ]:
LEN_FIR = 2048

## Make Source-Data

In [ ]:
A = np.finfo(np.float64).max / 2.
wave = np.sin(2. * np.pi * FREQ * t) + .5 * np.sin(2. * np.pi * 3. * FREQ * t)
rawdata = A * wave / 1.5

In [ ]:
plt.plot(t[:60], rawdata[:60])
plt.show()

# Main

In [ ]:
ssweep = SyncSweep(startfreq=10, stopfreq=RATE//2, durationappr=10, samplerate=RATE)
indata = np.array(ssweep, dtype=np.float64)
print(ssweep, indata.shape)

In [ ]:
plt.plot(ssweep)
plt.show()

In [ ]:
outdata = SOUND_FIELD(indata)
outdata.shape

In [ ]:
# hhir = HigherHarmonicImpulseResponse.from_sweeps(syncsweep=ssweep, measuredsweep=outdata)
# hhir
model = HammersteinModel.from_sweeps(syncsweep=ssweep, measuredsweep=outdata, orders=[1,])

In [ ]:
kernel = model.kernels[0]

In [ ]:
plt.plot(kernel.frf)
plt.show()

In [ ]:
FRFinv = 1. / kernel.frf
plt.plot(FRFinv)
plt.show()

In [ ]:
from scipy.fft import irfft
frf_inv = irfft(FRFinv)
plt.plot(frf_inv)
plt.show()

In [ ]:
from scipy.signal import convolve

In [ ]:
indata_modified = convolve(indata, frf_inv)

In [ ]:
outdata_modified = SOUND_FIELD(indata_modified)

In [ ]:
indata.shape, indata_modified.shape, outdata.shape, outdata_modified.shape

# Plot

In [ ]:
from scipy.fft import fft
plt.plot(20.*np.log10(np.abs(fft(frf_inv))), label='Impulse Response')
plt.xscale('log')
plt.xlabel('Frequency[Hz]')
plt.ylabel('Gain[dB]')
plt.legend()
plt.show()

In [ ]:
plt.plot(20.*np.log10(np.abs(fft(outdata_modified,))), label='Output FROM sound-field')
plt.plot(20.*np.log10(np.abs(fft(indata_modified,))), label='Output FOR sound-field')
plt.plot(20.*np.log10(np.abs(fft(indata,))), label='Input')
plt.xscale('log')
# plt.ylim([0, 100])
plt.xlabel('Frequency[Hz]')
plt.ylabel('Gain[dB]')
plt.legend()
plt.show()